# 🎬 B站/小红书视频字幕提取 - Colab版

仓库: https://github.com/easygl1der/my-bilisub

## 1️⃣ 挂载 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 项目路径
PROJECT_DIR = "/content/drive/MyDrive/my-projects/my-bilisub"
import os
os.chdir(PROJECT_DIR)

print(f"✅ 工作目录: {PROJECT_DIR}")
print(f"📁 文件列表:")
!ls -la

## 2️⃣ 安装依赖

In [ ]:
# 安装依赖
!pip install -q openai-whisper yt-dlp tqdm requests

# 检查 GPU
import torch
print(f"\n🔥 GPU 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 3️⃣ 配置参数

In [ ]:
# ============= 配置区域 =============

# 视频链接
VIDEO_URL = ""  # @param {type:"string"}

# Whisper 模型
WHISPER_MODEL = "small"  # @param ["tiny", "base", "small", "medium", "large"]

# 智谱 API（可选，用于字幕优化）
ZHIPU_API_KEY = ""  # @param {type:"string"}

# 是否启用 GLM 优化
ENABLE_GLM = False  # @param {type:"boolean"}

# =====================================

import os
if ZHIPU_API_KEY:
    os.environ['ZHIPU_API_KEY'] = ZHIPU_API_KEY

print(f"🎬 视频链接: {VIDEO_URL[:60] if VIDEO_URL else '(未设置)'}...")
print(f"📦 Whisper 模型: {WHISPER_MODEL}")
print(f"🤖 GLM 优化: {'启用' if ENABLE_GLM and ZHIPU_API_KEY else '禁用'}")

## 4️⃣ 运行 Whisper 转录

In [ ]:
import sys
import os
import time
import re
from pathlib import Path

import yt_dlp
import whisper

if not VIDEO_URL:
    print("❌ 请先设置 VIDEO_URL")
else:
    print("\n" + "="*60)
    print("🎬 开始处理视频")
    print("="*60)
    
    total_start = time.time()
    
    # 1. 下载音频
    print("\n⬇️ 步骤 1/2: 下载音频")
    print("-" * 40)
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'output/audio.%(ext)s',
        'quiet': True,
    }
    
    os.makedirs('output', exist_ok=True)
    
    start = time.time()
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(VIDEO_URL, download=True)
        title = info.get('title', 'unknown')
        duration = info.get('duration', 0)
    
    audio_file = list(Path('output').glob('audio.*'))[0]
    print(f"✅ 下载: {time.time()-start:.1f}秒 | 时长: {duration//60}分{duration%60}秒")
    print(f"📌 标题: {title}")
    
    # 2. Whisper 转录
    print("\n🎙️ 步骤 2/2: Whisper 转录")
    print("-" * 40)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🔧 设备: {device.upper()}")
    
    load_start = time.time()
    model = whisper.load_model(WHISPER_MODEL, device=device)
    print(f"✅ 模型加载: {time.time()-load_start:.1f}秒")
    
    transcribe_start = time.time()
    result = model.transcribe(str(audio_file), language='zh', verbose=False)
    transcribe_time = time.time() - transcribe_start
    
    print(f"✅ 转录完成: {transcribe_time:.1f}秒")
    
    # 保存结果
    safe_title = re.sub(r'[<>:"/\\|?*]', '_', title)[:80]
    output_dir = Path('output/transcripts')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 保存 SRT
    srt_path = output_dir / f"{safe_title}.srt"
    with open(srt_path, 'w', encoding='utf-8') as f:
        for i, seg in enumerate(result['segments'], 1):
            def fmt(t):
                h, m, s = int(t//3600), int((t%3600)//60), int(t%60)
                ms = int((t%1)*1000)
                return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"
            f.write(f"{i}\n{fmt(seg['start'])} --> {fmt(seg['end'])}\n{seg['text'].strip()}\n\n")
    
    # 保存 TXT
    txt_path = output_dir / f"{safe_title}.txt"
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(result['text'])
    
    total_time = time.time() - total_start
    
    print("\n" + "="*60)
    print("✅ 处理完成!")
    print("="*60)
    print(f"⏱️ 总耗时: {total_time:.1f}秒")
    print(f"📊 段落数: {len(result['segments'])}")
    print(f"\n📁 输出文件:")
    print(f"   SRT: {srt_path}")
    print(f"   TXT: {txt_path}")

## 5️⃣ GLM 字幕优化（可选）

In [ ]:
if ENABLE_GLM and ZHIPU_API_KEY:
    import requests
    from pathlib import Path
    
    # 获取最新 SRT 文件
    srt_files = list(Path('output/transcripts').glob('*.srt'))
    if not srt_files:
        print("❌ 未找到 SRT 文件，请先运行转录")
    else:
        srt_path = max(srt_files, key=lambda p: p.stat().st_mtime)
        print(f"📄 处理文件: {srt_path.name}")
        
        # 解析 SRT
        import re
        pattern = r'(\d+)\n(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.+?)(?=\n\n|$)'
        with open(srt_path, 'r', encoding='utf-8') as f:
            content = f.read()
        segments = re.findall(pattern, content, re.DOTALL)
        
        print(f"📊 共 {len(segments)} 个段落")
        
        BATCH_SIZE = 5
        optimized_segments = []
        
        for i in range(0, len(segments), BATCH_SIZE):
            batch = segments[i:i+BATCH_SIZE]
            batch_num = i // BATCH_SIZE + 1
            total_batches = (len(segments) + BATCH_SIZE - 1) // BATCH_SIZE
            
            combined_text = '\n'.join([s[3].strip() for s in batch])
            
            prompt = f"""请优化以下视频字幕：\n\n1. 修正错别字\n2. 添加标点\n3. 改善流畅度\n4. 保持原意\n\n原文（{len(batch)}行）：\n{combined_text}\n\n优化后（{len(batch)}行，每行一个）："""
            
            try:
                resp = requests.post(
                    "https://open.bigmodel.cn/api/paas/v4/chat/completions",
                    headers={
                        "Authorization": f"Bearer {ZHIPU_API_KEY}",
                        "Content-Type": "application/json"
                    },
                    json={
                        "model": "glm-4-flash",
                        "messages": [{"role": "user", "content": prompt}],
                        "temperature": 0.3
                    },
                    timeout=30
                )
                resp.raise_for_status()
                opt_text = resp.json()['choices'][0]['message']['content'].strip()
                opt_lines = opt_text.split('\n')
                
                for j, seg in enumerate(batch):
                    if j < len(opt_lines):
                        optimized_segments.append((seg[0], seg[1], seg[2], opt_lines[j].strip()))
                    else:
                        optimized_segments.append(seg)
                
                print(f"   批次 [{batch_num}/{total_batches}] 完成")
            except Exception as e:
                print(f"   批次 [{batch_num}/{total_batches}] 失败: {e}")
                optimized_segments.extend(batch)
        
        # 保存优化后的 SRT
        opt_srt_path = srt_path.parent / f"{srt_path.stem}_optimized.srt"
        with open(opt_srt_path, 'w', encoding='utf-8') as f:
            for seg in optimized_segments:
                f.write(f"{seg[0]}\n{seg[1]} --> {seg[2]}\n{seg[3]}\n\n")
        
        print(f"\n✅ 优化完成!")
        print(f"📄 输出: {opt_srt_path}")
else:
    print("⚠️ GLM 优化未启用")
    print("   如需启用，请设置 ENABLE_GLM=True 和 ZHIPU_API_KEY")

## 6️⃣ 下载结果

In [ ]:
from google.colab import files
from pathlib import Path

# 列出输出文件
output_dir = Path('output/transcripts')
if output_dir.exists():
    print("📁 可下载的文件:\n")
    for f in output_dir.glob('*'):
        size = f.stat().st_size / 1024
        print(f"  • {f.name} ({size:.1f} KB)")
    
    # 下载所有文件
    print("\n⬇️ 下载中...")
    for f in output_dir.glob('*'):
        files.download(str(f))
else:
    print("❌ 没有输出文件，请先运行转录")

## 📝 使用说明

### 快速开始
1. **启用 GPU**: 运行时 → 更改运行时类型 → T4 GPU
2. 设置 `VIDEO_URL` (你的 B站/小红书视频链接)
3. 按顺序运行所有单元格

### 可选功能
- **GLM 优化**: 设置 `ENABLE_GLM=True` 和 `ZHIPU_API_KEY`

### 模型选择
| 模型 | 大小 | 速度 | 准确度 |
|------|------|------|--------|
| tiny | ~1GB | 最快 | 一般 |
| base | ~1GB | 快 | 一般 |
| small | ~2GB | 较快 | 较好 |
| medium | ~5GB | 中等 | 好 |
| large | ~10GB | 慢 | 最好 |